# A Trader Stabilize Fiat Value of Crypto Currency

In [ ]:
TEST_RATIO = 1

In [ ]:
SAMPLE_INTERVAL = 60
HAPPY_COOLING_TIME = 60
SAD_COOLING_TIME = 60 * 6
BUY_RATE = 15
BUY_AMOUNT = 0.04 / TEST_RATIO
UNIT_CC_INIT_BUY_JPY = 40000
MAX_SELL_COUNT = 128
MAX_USED_JPY = 400000
MIN_UNIT_CC_GAIN_JPY = 400
MIN_UNIT_CC_LOSS_JPY = 400
MAX_UNIT_CC_LOSS_JPY = 4000
MAX_LOSS_JPY = 4000
MAX_TOTAL_LOSS_JPY = 8000

In [ ]:
from datetime import datetime
import json, sys, time
import python_bitbankcc

In [ ]:
pub = python_bitbankcc.public()

## Simulated One

In [ ]:
from unittest.mock import Mock
prv = Mock()

## Actual One

In [ ]:
# import os
# prv = python_bitbankcc.private(
#     os.environ['BITBANK_API_KEY'],
#     os.environ['BITBANK_API_SECRET']
# )

## Trading Operations

In [ ]:
def sell_all():
    now_ticker = pub.get_ticker('eth_jpy')
    now_sell_jpy = int(now_ticker.get('buy', '0'))
    now_buy_jpy = int(now_ticker.get('sell', '1'))

    please_sell_amount = BUY_AMOUNT * bought_unit_amount
    prv.order(
        pair='eth_jpy', price=str(now_sell_jpy), amount=str(please_sell_amount), side='sell', order_type='market'
    )

    bought_amount = BUY_AMOUNT * bought_unit_amount
    total_gain_jpy = total_gained_jpy + now_sell_jpy * bought_amount - used_jpy
    with open('sample-history.txt', 'a', 1) as fsh:
        fsh.write(
            f'BUY ETH for {now_buy_jpy} JPY: SELL ETH for {now_sell_jpy} JPY\n'
            f'GAIN {total_gain_jpy} JPY in total at {datetime.now()}\n\n'
        )
    print(
        f'BUY ETH for {now_buy_jpy} JPY: SELL ETH for {now_sell_jpy} JPY\n'
        f'GAIN {total_gain_jpy} JPY in total at {datetime.now()}\n\n',
        end=''
    )

    avg_jpy = used_jpy / bought_amount
    diff_jpy = now_sell_jpy - avg_jpy
    gained_jpy = diff_jpy * please_sell_amount
    with open('transaction-history.txt', 'a', 1) as fth:
        fth.write(
            f'SOLD {please_sell_amount:.2f}=+{bought_amount:.2f}({used_jpy/MAX_USED_JPY*100:.2f}%)'
            f'-{bought_amount-please_sell_amount:.2f}(0.00%) ETH'
            f' for {diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY：BUY ETH for {now_buy_jpy} JPY\n'
            f'GAINED {gained_jpy} JPY: GAINED {total_gained_jpy + gained_jpy} JPY in total at {datetime.now()}\n\n'
        )
    print(
        f'SOLD {please_sell_amount:.2f}=+{bought_amount:.2f}({used_jpy/MAX_USED_JPY*100:.2f}%)'
        f'-{bought_amount-please_sell_amount:.2f}(0.00%) ETH'
        f' for {diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY：BUY ETH for {now_buy_jpy} JPY\n'
        f'GAINED {gained_jpy} JPY: GAINED {total_gained_jpy + gained_jpy} JPY in total at {datetime.now()}\n\n',
        end=''
    )

    return {
        'sample_number': sample_number,
        'buy_count': buy_count,
        'sell_count': sell_count,
        'bought_unit_amount': bought_unit_amount,
        'used_jpy': used_jpy,
        'please_buy_unit_amount': please_buy_unit_amount,
        'please_sell_unit_amount': please_sell_unit_amount,
        'cooling_time': cooling_time,
        'total_gained_jpy': total_gained_jpy
    }

## Start Trading

In [ ]:
sample_number = 0
buy_count = 0
sell_count = 0
bought_unit_amount = 0
used_jpy = 0
bought_average_jpy = UNIT_CC_INIT_BUY_JPY
please_buy_unit_amount = 0
please_sell_unit_amount = 0
cooling_time = HAPPY_COOLING_TIME
total_gained_jpy = 0

In [ ]:
with open('sample-history.txt', 'a', 1) as fsh, open('transaction-history.txt', 'a', 1) as fth:
    while True:
        now_ticker = pub.get_ticker('eth_jpy')
        now_sell_jpy = int(now_ticker.get('buy', '0'))
        now_buy_jpy = int(now_ticker.get('sell', '1'))

        bought_amount = BUY_AMOUNT * bought_unit_amount

        if sample_number % BUY_RATE == 0:
            if bought_unit_amount > 0 and now_sell_jpy < used_jpy/bought_amount - MAX_UNIT_CC_LOSS_JPY:
                please_sell_unit_amount = bought_unit_amount
                cooling_time = SAD_COOLING_TIME

            if now_sell_jpy * bought_amount - used_jpy < -MAX_LOSS_JPY:
                please_sell_unit_amount = bought_unit_amount
                cooling_time = SAD_COOLING_TIME

        if sample_number % BUY_RATE == 0 and bought_unit_amount > 0 and now_sell_jpy > used_jpy/bought_amount:
            please_sell_unit_amount = 1
            cooling_time = HAPPY_COOLING_TIME

        if bought_unit_amount > 0 and now_sell_jpy > used_jpy/bought_amount + MIN_UNIT_CC_GAIN_JPY \
                * (MAX_USED_JPY ** 2 - used_jpy ** 2) ** 0.5 / MAX_USED_JPY:
            sample_number = 0
            please_sell_unit_amount = (bought_unit_amount + 1) // 2
            cooling_time = HAPPY_COOLING_TIME

        if please_sell_unit_amount > 0:
            please_sell_amount = BUY_AMOUNT * please_sell_unit_amount
            prv.order(
                pair='eth_jpy', price=str(now_sell_jpy), amount=str(please_sell_amount), side='sell', order_type='market'
            )

            bought_amount = BUY_AMOUNT * bought_unit_amount
            total_gain_jpy = total_gained_jpy + now_sell_jpy * bought_amount - used_jpy
            fsh.write(
                f'BUY ETH for {now_buy_jpy} JPY: SELL ETH for {now_sell_jpy} JPY\n'
                f'GAIN {total_gain_jpy} JPY in total at {datetime.now()}\n\n'
            )

            used_jpy_after = used_jpy * (bought_unit_amount - please_sell_unit_amount) / bought_unit_amount

            avg_jpy = used_jpy / bought_amount
            diff_jpy = now_sell_jpy - avg_jpy
            gained_jpy = diff_jpy * please_sell_amount
            total_gained_jpy += gained_jpy
            fth.write(
                f'SOLD {please_sell_amount:.2f}=+{bought_amount:.2f}({used_jpy/MAX_USED_JPY*100:.2f}%)'
                f'-{bought_amount-please_sell_amount:.2f}({used_jpy_after/MAX_USED_JPY*100:.2f}%) ETH'
                f' for {diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY：BUY ETH for {now_buy_jpy} JPY\n'
                f'GAINED {gained_jpy} JPY: GAINED {total_gained_jpy} JPY in total at {datetime.now()}\n\n'
            )

            buy_count = 0
            sell_count += 1
            bought_unit_amount -= please_sell_unit_amount
            used_jpy *= bought_unit_amount / (bought_unit_amount + please_sell_unit_amount)
            please_buy_unit_amount = 0
            please_sell_unit_amount = 0

            if sell_count == MAX_SELL_COUNT or total_gained_jpy < -MAX_TOTAL_LOSS_JPY:
                sell_count = 0
                cooling_time = HAPPY_COOLING_TIME
                total_gained_jpy = 0
                break

            time.sleep(cooling_time/TEST_RATIO)
            sample_number += 1
            cooling_time = HAPPY_COOLING_TIME
            continue

        if sample_number % BUY_RATE == 0 and now_buy_jpy < bought_average_jpy \
                and used_jpy + now_buy_jpy * BUY_AMOUNT <= MAX_USED_JPY:
            please_buy_unit_amount = 1

        if bought_unit_amount > 0 and now_buy_jpy < used_jpy/bought_amount - MIN_UNIT_CC_LOSS_JPY \
                * (used_jpy * MAX_USED_JPY * 2 - used_jpy ** 2) ** 0.5 / MAX_USED_JPY:
            buy_unit_amount = bought_unit_amount
            while buy_unit_amount > 0 and used_jpy + now_buy_jpy * BUY_AMOUNT * buy_unit_amount > MAX_USED_JPY:
                buy_unit_amount >>= 1
            if buy_unit_amount > 0:
                sample_number = 0
                please_buy_unit_amount = buy_unit_amount

        if please_buy_unit_amount > 0:
            please_buy_amount = BUY_AMOUNT * please_buy_unit_amount
            prv.order(
                pair='eth_jpy', price=str(now_buy_jpy), amount=str(please_buy_amount), side='buy', order_type='market'
            )

            used_jpy_before = used_jpy

            buy_count += 1
            bought_unit_amount += please_buy_unit_amount
            used_jpy += now_buy_jpy * please_buy_amount
            bought_average_jpy = used_jpy / (BUY_AMOUNT * bought_unit_amount)
            please_buy_unit_amount = 0

            bought_amount = BUY_AMOUNT * bought_unit_amount
            avg_jpy = used_jpy / bought_amount
            diff_jpy = now_sell_jpy - avg_jpy
            gain_jpy = now_sell_jpy * bought_amount - used_jpy
            total_gain_jpy = total_gained_jpy + gain_jpy
            fth.write(
                f'BOUGHT {please_buy_amount:.2f}=-{bought_amount-please_buy_amount:.2f}({used_jpy_before/MAX_USED_JPY*100:.2f}%)'
                f'+{bought_amount:.2f}({used_jpy/MAX_USED_JPY*100:.2f}%) ETH for {now_buy_jpy} JPY:'
                f' SELL {bought_amount:.2f} for {diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY: NO.{buy_count} at {datetime.now()}\n'
                f'GAIN {gain_jpy} JPY: GAIN {total_gain_jpy} JPY in total at {datetime.now()}\n\n'
            )
        elif sample_number % BUY_RATE == 0:
            bought_amount = BUY_AMOUNT * bought_unit_amount
            total_gain_jpy = total_gained_jpy + now_sell_jpy * bought_amount - used_jpy
            fsh.write(
                f'BUY ETH for {now_buy_jpy} JPY: SELL ETH for {now_sell_jpy} JPY\n'
                f'GAIN {total_gain_jpy} JPY in total at {datetime.now()}\n\n'
            )

            bought_amount = BUY_AMOUNT * bought_unit_amount
            avg_jpy = bought_average_jpy
            diff_jpy = now_sell_jpy - avg_jpy
            total_gain_jpy = total_gained_jpy + now_sell_jpy * bought_amount - used_jpy
            fth.write(
                f'BUY for {now_buy_jpy} JPY: SELL for {diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY: at {datetime.now()}\n'
                f'GAIN {total_gain_jpy} JPY in total at {datetime.now()}\n\n'
            )

            time.sleep(SAMPLE_INTERVAL / TEST_RATIO)
            continue

        bought_amount = BUY_AMOUNT * bought_unit_amount
        total_gain_jpy = total_gained_jpy + now_sell_jpy * bought_amount - used_jpy
        fsh.write(
            f'BUY ETH for {now_buy_jpy} JPY: SELL ETH for {now_sell_jpy} JPY\n'
            f'GAIN {total_gain_jpy} JPY in total at {datetime.now()}\n\n'
        )

        time.sleep(SAMPLE_INTERVAL / TEST_RATIO)
        sample_number += 1

In [ ]:
sell_all()